# Лабораторная работа 7. Управление ресурсами в однопроцессорной системе с неоднородными заявками
Выполнил: Пакало Александр Сергеевич, студент РТ5-81Б

## Задание 1
В однопроцессорную систему случайным образом поступают на выполнение $m$
разных типов программ, отличающихся известной трудоемкостью
$Q_1, Q_2,\dots, Q_m$.
Входящий поток простейший с интенсивностью $\lambda$.

Представив данную систему как одноканальную СМО с неограниченной очередью,
вычислить среднее время обслуживания программ, считая длительность
обслуживания случайной величиной (теоретически и экспериментально).

Реализовать алгоритм SPT, выбирая из очереди заявки в соответствии с их
приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ.
Сравнить полученные результаты.

Реализовать алгоритм RR при заданном кванте времени $q$. Оценить среднее время
обслуживания программ. Сравнить полученные результаты. Выяснить, как влияет
величина кванта на среднее время обслуживания программ.

In [1]:
Variant <- 5
set.seed(Variant)
m <- sample(c(6:20), 1)
lambda <- runif(1, 0.1, 2)
Q <- rexp(m, 0.3)
q <- sample(c(1:4), 1)
View(data.frame(m, q, lambda))
print(Q)

m,q,lambda
<int>,<int>,<dbl>
7,1,1.401915


[1] 0.2417931 1.3403831 0.1863999 2.0529013 0.2552007 2.9172096 6.6709868


Заведем таблицу результатов

In [59]:
results <- data.frame(0, 0, 0, 0, 0)
colnames(results) <- c("M/M/1/infty theoretical", "M/M/1/infty practical", "SPT theoretical", "SPT practical", "Round Robin")

### СМО вида $М/М/1/\infty$
Представим данную систему как одноканальную СМО с неограниченной очередью.

#### Теоретически

In [72]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [73]:
y <- lambda / mu
y

[1] 2.736714

Так как $y > 1$, поменяем $\lambda$.

In [75]:
lambda <- 0.3

In [76]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [77]:
y <- lambda / mu
y

[1] 0.5856375

In [78]:
results[1] <- 1 / mu / (1 - y)
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.711152,3.662282,4.008172,2.5725,2.5725


#### Численно

In [111]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [112]:
programs <- trajectory("programs' path") %>%
    seize("server", amount = 1) %>%
    timeout(function() rexp(1, mu)) %>%
    release("server", amount = 1)

In [113]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", 1) %>%
    add_generator("programs", programs, function() rexp(1, lambda)) %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperSim | now: 10000 | next: 10001.2542877312
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 1(1) | queue status: 0(Inf) }
{ Source: programs | monitored: 1 | n_generated: 2942 }

In [114]:
env %>%
    get_mon_resources()

resource,time,server,queue,capacity,queue_size,system,limit,replication
<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
server,4.263362,1,0,1,Inf,1,Inf,1
server,5.042783,0,0,1,Inf,0,Inf,1
server,6.746470,1,0,1,Inf,1,Inf,1
server,7.256152,0,0,1,Inf,0,Inf,1
server,7.619343,1,0,1,Inf,1,Inf,1
server,7.906546,0,0,1,Inf,0,Inf,1
server,8.957836,1,0,1,Inf,1,Inf,1
server,9.077252,1,1,1,Inf,2,Inf,1
server,9.511587,1,2,1,Inf,3,Inf,1


In [115]:
arrivals <- env %>%
    get_mon_arrivals()
arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs0,4.263362,5.042783,0.779421017,TRUE,1
programs1,6.746470,7.256152,0.509681929,TRUE,1
programs2,7.619343,7.906546,0.287202954,TRUE,1
programs3,8.957836,11.773471,2.815635448,TRUE,1
programs4,9.077252,11.812946,0.039474524,TRUE,1
programs5,9.511587,13.207806,1.394859912,TRUE,1
programs6,10.964045,13.508030,0.300223750,TRUE,1
programs7,11.346586,14.121309,0.613279763,TRUE,1
programs8,15.411978,15.416274,0.004295376,TRUE,1


In [116]:
results[2] <- mean(arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.545182,4.008172,2.5725,2.5725


### Алгоритм SPT

#### Теоретически
Среднее время обслуживания складывается из ожидания в очереди и времени
выполнения, усредненным по всем заявкам:
$$
T_{\text{сист}} = \frac{1}{m} \left( Q'_1 + \left( Q'_1 + Q'_2 \right) +
\left(Q'_1 + Q'_2 + Q'_3 \right) + \dots + \sum^m_{i = 1}Q'_i \right)
$$
где $Q'_i$ - $i$-й элемент массива $Q'$,
**отсортированного по возрастанию** массива Q.

In [117]:
Q_sorted <- sort(Q)
Q_sorted

[1] 0.1863999 0.2417931 0.2552007 1.3403831 2.0529013 2.9172096 6.6709868

Суммы первых i элементов:

In [118]:
Q_progression_sums <- lapply(
    seq_along(Q_sorted),
    function(i) sum(head(Q_sorted, i))
)
Q_progression_sums

[[1]]
[1] 0.1863999

[[2]]
[1] 0.428193

[[3]]
[1] 0.6833937

[[4]]
[1] 2.023777

[[5]]
[1] 4.076678

[[6]]
[1] 6.993888

[[7]]
[1] 13.66487

Итоговая сумма

In [4]:
sum_of_Q_progression_sums <- sum(unlist(Q_progression_sums))
sum_of_Q_progression_sums

[1] 28.0572

In [119]:
results[3] <- 1 / m * sum_of_Q_progression_sums
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.545182,4.008172,2.5725,2.5725


#### Численно

In [120]:
N <- 10000
programs <- sample(Q, N, replace = TRUE)
programs

[1] 0.2552007 2.0529013 2.0529013 0.1863999 6.6709868 2.9172096 6.6709868
    [8] 2.0529013 0.1863999 0.2552007 2.9172096 6.6709868 2.0529013 6.6709868
   [15] 0.2417931 2.0529013 1.3403831 2.0529013 1.3403831 6.6709868 6.6709868
   [22] 2.9172096 0.2552007 2.9172096 2.0529013 0.1863999 6.6709868 2.9172096
   [29] 0.2417931 2.9172096 0.1863999 2.0529013 0.2417931 0.2417931 0.1863999
   [36] 2.0529013 1.3403831 0.1863999 2.9172096 0.1863999 0.2417931 2.9172096
   [43] 0.2552007 0.2417931 0.1863999 6.6709868 2.0529013 0.2417931 1.3403831
   [50] 6.6709868 0.1863999 2.0529013 6.6709868 1.3403831 6.6709868 1.3403831
   [57] 0.2552007 0.2552007 2.9172096 6.6709868 2.9172096 0.1863999 1.3403831
   [64] 2.9172096 6.6709868 1.3403831 0.1863999 2.9172096 0.1863999 0.2552007
   [71] 1.3403831 0.1863999 2.9172096 1.3403831 0.2417931 0.2417931 0.1863999
   [78] 0.1863999 2.0529013 6.6709868 0.2417931 2.0529013 2.0529013 2.0529013
   [85] 0.2552007 0.2417931 6.6709868 0.2552007 1.3403831 0.2552007 6.6709868
   [92] 2.0529013 0.2552007 1.3403831 2.9172096 0.2552007 0.2417931 2.9172096
   [99] 6.6709868 2.0529013 0.1863999 6.6709868 2.0529013 0.2552007 6.6709868
  [106] 1.3403831 2.9172096 0.2552007 1.3403831 1.3403831 0.2417931 6.6709868
  [113] 6.6709868 6.6709868 0.1863999 6.6709868 1.3403831 0.2552007 0.2417931
  [120] 2.0529013 0.2552007 0.2552007 1.3403831 0.2552007 1.3403831 6.6709868
  [127] 2.9172096 2.9172096 0.2417931 0.2417931 6.6709868 1.3403831 0.1863999
  [134] 6.6709868 0.2417931 6.6709868 0.2417931 0.1863999 0.2417931 0.2417931
  [141] 0.2417931 0.2552007 2.0529013 2.9172096 0.2552007 0.2417931 0.1863999
  [148] 1.3403831 0.2417931 1.3403831 1.3403831 0.2552007 2.0529013 1.3403831
  [155] 6.6709868 0.1863999 6.6709868 0.2552007 2.0529013 0.2552007 0.1863999
  [162] 0.1863999 0.2417931 0.2552007 6.6709868 2.0529013 0.2552007 0.1863999
  [169] 6.6709868 0.2417931 1.3403831 0.2417931 6.6709868 6.6709868 2.9172096
  [176] 0.2417931 6.6709868 2.0529013 0.2417931 2.0529013 6.6709868 0.2552007
  [183] 1.3403831 2.9172096 6.6709868 1.3403831 2.0529013 0.2417931 6.6709868
  [190] 6.6709868 0.1863999 0.2417931 2.9172096 1.3403831 0.1863999 0.1863999
  [197] 1.3403831 2.9172096 2.9172096 0.2417931 0.2417931 1.3403831 1.3403831
  [204] 2.9172096 0.2552007 2.9172096 1.3403831 0.1863999 1.3403831 2.0529013
  [211] 2.0529013 0.1863999 0.1863999 0.2552007 2.0529013 2.0529013 6.6709868
  [218] 2.0529013 1.3403831 0.1863999 1.3403831 0.2552007 2.0529013 0.2552007
  [225] 2.9172096 1.3403831 2.9172096 6.6709868 1.3403831 0.1863999 0.2552007
  [232] 0.2552007 0.1863999 1.3403831 2.9172096 0.2552007 2.9172096 2.0529013
  [239] 1.3403831 0.1863999 1.3403831 1.3403831 0.1863999 0.2417931 0.2417931
  [246] 2.9172096 6.6709868 1.3403831 1.3403831 2.0529013 0.2417931 2.9172096
  [253] 2.0529013 6.6709868 1.3403831 6.6709868 0.1863999 2.9172096 2.9172096
  [260] 0.1863999 2.9172096 2.9172096 2.9172096 6.6709868 1.3403831 0.1863999
  [267] 2.0529013 0.2417931 2.0529013 1.3403831 2.0529013 0.1863999 0.1863999
  [274] 2.0529013 2.9172096 6.6709868 2.0529013 1.3403831 6.6709868 0.1863999
  [281] 0.2417931 6.6709868 0.2552007 0.1863999 1.3403831 2.0529013 2.9172096
  [288] 6.6709868 1.3403831 6.6709868 0.1863999 1.3403831 0.1863999 2.0529013
  [295] 0.2417931 0.2417931 6.6709868 0.2417931 0.1863999 1.3403831 0.2552007
  [302] 2.9172096 2.9172096 0.2552007 2.9172096 0.2552007 0.2552007 1.3403831
  [309] 0.2417931 0.1863999 0.2552007 2.0529013 0.1863999 0.2417931 0.2552007
  [316] 0.2552007 2.9172096 1.3403831 0.2552007 6.6709868 0.2417931 0.2552007
  [323] 2.9172096 0.2552007 2.0529013 0.1863999 1.3403831 0.2417931 0.1863999
  [330] 2.9172096 0.1863999 1.3403831 0.2552007 0.1863999 2.9172096 1.3403831
  [337] 0.2552007 2.9172096 1.3403831 2.0529013 2.0529013 2.0529013 0.2552007
  [344] 1.3403831 1.3403831 2.0529013 2.0529013 0.2552007 0.2417931 6.6709868
  [351] 0.2417931 0.1863999 0.2552007 1.3403831 0.2552007 0.2417931 2.9172096
  [358] 0.1863999 0.255200

In [121]:
time <- 0

task_schedule <- sort(programs)

while (length(task_schedule) > 0) {
    time <- time + q
    task_schedule[1] <- task_schedule[1] - q

    if (task_schedule[1] <= 0) {
        task_schedule <- tail(task_schedule, length(task_schedule) - 1)
    }
}

results[4] <- time / N
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.545182,4.008172,2.5598,2.5725


### Алгоритм Round Robin
Реализуем round robin.

In [70]:
time <- 0

task_schedule <- programs

while (length(task_schedule) > 0) {
    time <- time + q
    task_schedule[1] <- task_schedule[1] - q

    if (task_schedule[1] <= 0) {
        task_schedule <- tail(task_schedule, length(task_schedule) - 1)
    }
}

results[5] <- time / N
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5.887677,3.662282,4.008172,2.5725,2.5725


In [10]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

Loading required package: simmer



simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [41]:
programs <- trajectory("programs' path") %>%
    seize("server", amount = 1) %>%
    timeout(function() rexp(1, 1)) %>%
    release("server", amount = 1)

In [42]:
programs <- trajectory("programs' path") %>%
    select("server", "round-robin") %>%
    seize_selected(1) %>%
    timeout(function() rexp(1, 1)) %>%
    release_selected(1)

In [43]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", 1) %>%
    add_generator("programs", programs, function() rexp(1, 1)) %>%
    run(until = SIMULATION_TIME)

Warning message in add_resource_(.env$sim_obj, i, positive(capacity), positive(queue_size), :
“resource 'server' already defined”
Warning message in add_generator_(.env$sim_obj, i, trajectory, distribution, mon, :
“process 'programs' already defined”


simmer environment: SuperDuperSim | now: 10000 | next: 10001.4418565715
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 1(1) | queue status: 4311(Inf) }
{ Source: programs | monitored: 1 | n_generated: 7042 }

In [ ]:
env() %>%
    get_mon_resources()